In [2]:
from pyspark.sql import SparkSession, functions as F
import pandas as pd
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Interaction
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
import plotly.express as px
import datetime

In [41]:
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.driver.memory", '8g')
    .config("spark.executor.memory", '8g')
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.parquet.enableVectorizedReader","false")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/10/10 06:51:38 WARN Utils: Your hostname, DESKTOP-3NQ3PQI resolves to a loopback address: 127.0.1.1; using 172.17.18.177 instead (on interface eth0)
22/10/10 06:51:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 06:51:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 06:51:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
merchants_df = pd.read_parquet('../data/curated/merchants.parquet')
predictions_df = pd.read_parquet('../data/curated/lr_transaction_predictions.parquet')
fraud_df = pd.read_csv('../data/tables/merchant_fraud_probability.csv')

transactions_sdf = spark.read.parquet(
    '../data/tables/transactions_20210228_20210827_snapshot'
).union(
    spark.read.parquet(
        '../data/tables/transactions_20210828_20220227_snapshot'
    )
).union(
    spark.read.parquet(
        '../data/tables/transactions_20220228_20220828_snapshot'
    )
)

In [33]:
total_predictions_df = pd.merge(
    predictions_df.groupby('merchant_abn')['weighted_dollar_value'].sum(),
    merchants_df[['merchant_abn', 'take_rate']],
    on = 'merchant_abn'
)

total_predictions_df = pd.merge(
    total_predictions_df,
    fraud_df.groupby('merchant_abn').sum(),
    on = 'merchant_abn',
    how = 'left'
).fillna(0)

total_predictions_df = pd.merge(
    total_predictions_df,
    transactions_sdf.groupby('merchant_abn').count().toPandas(),
    on = 'merchant_abn'
)

In [45]:
total_predictions_df = total_predictions_df.sort_values(by = 'weighted_dollar_value', ascending=False)

In [46]:
total_predictions_df[:100].mean()

merchant_abn             5.884611e+10
weighted_dollar_value    4.417911e+04
take_rate                5.078800e-02
fraud_probability        1.301231e+01
count                    4.672841e+04
dtype: float64

In [47]:
total_predictions_df[100:].mean()

merchant_abn             5.436173e+10
weighted_dollar_value    2.713707e+03
take_rate                4.381429e-02
fraud_probability        6.147805e-01
count                    2.285154e+03
dtype: float64

In [44]:
total_predictions_df

,merchant_abn,weighted_dollar_value,take_rate,fraud_probability,count
0,79827781481,84725.637293,0.0682,59.470319,4798
1,32361057556,83900.069215,0.0661,0.000000,85780
2,48534649627,83381.611296,0.0664,85.703799,66437
3,86578477987,80930.979363,0.0643,0.000000,272674
4,38700038932,80746.407576,0.0631,0.000000,7132
...,...,...,...,...,...
4008,54277261175,664.479120,0.0146,0.000000,6
4009,55622580330,661.760494,0.0010,0.000000,33
4010,19340071232,660.607932,0.0360,0.000000,5
4011,55555661470,659.879296,0.0183,0.000000,4


In [ ]:
total_predictions_df = predictions_df.groupby(
    'merchant_abn'
)['weighted_dollar_value'].sum().reset_index()

In [ ]:
px.histogram(total_predictions_df, 'weighted_dollar_value')

In [ ]:
px.scatter(pd.merge(total_predictions_df, merchants_df, on = 'merchant_abn'), x = 'take_rate', y = 'weighted_dollar_value', trendline = 'ols', hover_data = ['merchant_abn'])

In [ ]:
total_predictions_df['band'] = total_predictions_df['weighted_dollar_value'] < 20000

In [ ]:
predictions_df = pd.merge(
    predictions_df,
    total_predictions_df[['merchant_abn', 'band']],
    on = 'merchant_abn'
)

In [ ]:
px.scatter(predictions_df.groupby(['order_datetime', 'band'])['weighted_dollar_value'].mean().reset_index(), x = 'order_datetime', y = 'weighted_dollar_value', color = 'band')

In [ ]:
predictions_df